In [ ]:
import mne
import os

BASE_DIR = r"rEEG"
SUBJ = "sub-121"

raw_path = os.path.join(BASE_DIR, SUBJ, "postICA121_raw.fif")
raw = mne.io.read_raw_fif(raw_path, preload=True)

# 2-second epochs with 50% overlap
events = mne.make_fixed_length_events(raw, duration=2.0, overlap=1.0)
reject_criteria = dict(eeg=200e-6)  # relaxed threshold (200 µV)

epochs = mne.Epochs(raw, events=events, tmin=0, tmax=2.0,
                    baseline=None, reject=reject_criteria, preload=True)

# Remove subjects with <20 clean epochs
if len(epochs) < 20:
    print(f"{SUBJ} has too few clean epochs ({len(epochs)}), skipping.")
else:
    EPOCH_OUT_NAME = "epo_121_raw.fif"
    epochs.save(os.path.join(BASE_DIR, SUBJ, EPOCH_OUT_NAME), overwrite=True)
    print(f"{SUBJ}: {len(epochs)} clean epochs saved → {EPOCH_OUT_NAME}")


Opening raw data file rEEG\sub-078\postICA078_raw.fif...
    Range : 0 ... 36464 =      0.000 ...   145.856 secs
Ready.
Reading 0 ... 36464  =      0.000 ...   145.856 secs...
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 144 events and 501 original time points ...
0 bad epochs dropped
sub-078: 144 clean epochs saved → epo_059_raw.fif


C:\Users\User\AppData\Local\Temp\ipykernel_42796\721354534.py:22: RuntimeWarning: This filename (rEEG\sub-078\epo_059_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(os.path.join(BASE_DIR, SUBJ, EPOCH_OUT_NAME), overwrite=True)


In [3]:
import mne
import os
import numpy as np

# -------------------------
# CONFIG
# -------------------------
BASE_DIR = r"rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(46,150)]  # adjust range
EPOCH_DURATION = 2.0  # seconds
OVERLAP = 0.5         # 50%
REJECT_THRESHOLD = dict(eeg=200e-6)  # 150 µV for EEG channels
MIN_EPOCHS = 20

RAW_NAME = "postICA{sid}_raw.fif"
EPOCH_OUT_NAME = "epo_{sid}_raw.fif"

# -------------------------
# LOOP OVER SUBJECTS
# -------------------------
for subj in SUBJECTS:
    sid = subj.split("-")[1]
    print(f"\n=== Processing {subj} ===")

    raw_path = os.path.join(BASE_DIR, subj, RAW_NAME.format(sid=sid))
    out_path = os.path.join(BASE_DIR, subj, EPOCH_OUT_NAME.format(sid=sid))

    if not os.path.exists(raw_path):
        print("Missing raw file — skipping")
        continue

    # Load cleaned raw
    raw = mne.io.read_raw_fif(raw_path, preload=True)

    # Create overlapping events for continuous data
    step = EPOCH_DURATION * (1 - OVERLAP)
    n_steps = int((raw.times[-1] - EPOCH_DURATION) // step) + 1
    events = []
    for i in range(n_steps):
        sample = int(i * step * raw.info['sfreq'])
        events.append([sample, 0, 1])
    events = np.array(events)

    # Epoching
    epochs = mne.Epochs(
        raw, events=events, tmin=0, tmax=EPOCH_DURATION,
        baseline=None, preload=True, reject=REJECT_THRESHOLD
    )

    # Check minimum epochs
    if len(epochs) < MIN_EPOCHS:
        print(f"Subject {subj} has only {len(epochs)} clean epochs — skipping")
        continue

    # Save epochs
    epochs.save(out_path, overwrite=True)
    print(f"{subj}: {len(epochs)} clean epochs saved → {out_path}")

print("\n=== ALL SUBJECTS PROCESSED ===")



=== Processing sub-046 ===
Opening raw data file rEEG\sub-046\postICA046_raw.fif...
    Range : 0 ... 32819 =      0.000 ...   131.276 secs
Ready.
Reading 0 ... 32819  =      0.000 ...   131.276 secs...
Not setting metadata
130 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 130 events and 501 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
sub-046: 130 clean epochs saved → rEEG\sub-046\epo_046_raw.fif

=== Processing sub-047 ===
Opening raw data file rEEG\sub-047\postICA047_raw.fif...
    Range : 0 ... 32734 =      0.000 ...   130.936 secs
Ready.
Reading 0 ... 32734  =      0.000 ...   130.936 secs...
Not setting metadata
129 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 129 events and 501 original time points ...
0 bad epochs dropped


C:\Users\User\AppData\Local\Temp\ipykernel_35480\2407947738.py:56: RuntimeWarning: This filename (rEEG\sub-046\epo_046_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_path, overwrite=True)


Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
sub-047: 129 clean epochs saved → rEEG\sub-047\epo_047_raw.fif

=== Processing sub-048 ===
Opening raw data file rEEG\sub-048\postICA048_raw.fif...
    Range : 0 ... 59504 =      0.000 ...   238.016 secs
Ready.
Reading 0 ... 59504  =      0.000 ...   238.016 secs...


C:\Users\User\AppData\Local\Temp\ipykernel_35480\2407947738.py:56: RuntimeWarning: This filename (rEEG\sub-047\epo_047_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_path, overwrite=True)


Not setting metadata
237 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 237 events and 501 original time points ...
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'C3', 'F8', 'AF7', 'F5', 'FT7', 'FC3', 'C5', 'F6']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'C3', 'TP9', 'FT10', 'F8', 'AF7', 'F5', 'FT7', 'FC3', 'C5', 'F6']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'C3', 'TP9', 'TP10', 'FT10', 'FC6', 'F8', 'AF7', 'F5', 'FT7', 'FC3', 'C5', 'FT8', 'F6', 'AF8']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'C3', 'TP9', 'TP10', 'F8', 'F5', 'FT7', 'FC3', 'C5', 'FT8', 'F6', 'AF8']
    Rejecting  epoch based on EEG : ['FC5']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'FT7', 'F6']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'TP9', 'FT10', 'FC6', 'F8', 'AF7', 'F5', 'FT7', 'FC3', 'C5', 'FT8', 'F6', 'AF8']
    Rejecting  epoch based on EEG : ['F7', 'FC5', 'TP9', 'FT10', 'FC6', 'F8', 'AF7', 'F5', '

...

C:\Users\User\AppData\Local\Temp\ipykernel_35480\2407947738.py:56: RuntimeWarning: This filename (rEEG\sub-147\epo_147_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_path, overwrite=True)


0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
sub-148: 157 clean epochs saved → rEEG\sub-148\epo_148_raw.fif

=== Processing sub-149 ===
Opening raw data file rEEG\sub-149\postICA149_raw.fif...


C:\Users\User\AppData\Local\Temp\ipykernel_35480\2407947738.py:56: RuntimeWarning: This filename (rEEG\sub-148\epo_148_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_path, overwrite=True)


    Range : 0 ... 32104 =      0.000 ...   128.416 secs
Ready.
Reading 0 ... 32104  =      0.000 ...   128.416 secs...
Not setting metadata
127 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 127 events and 501 original time points ...
0 bad epochs dropped
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
sub-149: 127 clean epochs saved → rEEG\sub-149\epo_149_raw.fif

=== ALL SUBJECTS PROCESSED ===


C:\Users\User\AppData\Local\Temp\ipykernel_35480\2407947738.py:56: RuntimeWarning: This filename (rEEG\sub-149\epo_149_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_path, overwrite=True)
